In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Escaladores
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder


In [ ]:
df = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_to_encode.csv')

OneHotEncoder

In [ ]:
df_encoded = df.copy()

encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_features = encoder.fit_transform(df_encoded[['property_types']])
# Verifica el número de características generadas
feature_names = encoder.get_feature_names_out()
print(f'Number of feature names: {len(feature_names)}')
print(f'Number of encoded features: {encoded_features.shape[1]}')

# Crear el DataFrame con las características codificadas
encoded_df = pd.DataFrame(encoded_features, columns=feature_names)

He cambiado los nombres para que nos resulte más fácil

In [ ]:
encoded_df = encoded_df.rename(columns={'property_types_Habitación': 'habitacion'})
encoded_df = encoded_df.rename(columns={'property_types_otro': 'alojamiento entero'})
encoded_df.head()

Lo unimos todo y dropeamos property_types

In [ ]:
df_encoded = pd.concat([df_encoded, encoded_df], axis=1)
df_encoded.head()

In [ ]:
df_encoded = df_encoded.drop('property_types',axis= 1)

In [ ]:
df_encoded['habitacion'] = df_encoded['habitacion'].astype('int64')

In [ ]:
df_encoded['alojamiento entero'] = df_encoded['alojamiento entero'].astype('int64')

KNNImputer

In [ ]:
%%time
imputer = KNNImputer(n_neighbors=3)
df_encoded = pd.DataFrame(imputer.fit_transform(df_encoded), columns=df_encoded.columns)
df_encoded

Creamos una copia

In [ ]:
df = df_encoded.copy()

HeatMap

In [ ]:
plt.figure(figsize=(30, 20))

sns.heatmap(data = df_encoded.corr(), annot = True,cmap="Blues")

Feature Importance

In [ ]:
%%time

# Feature Importance

# Modelo de RandomForest para obtener Feature Importance
model = RandomForestRegressor(n_estimators = 250,
                               random_state = 0)
# Entrenamos el modelo
model.fit(df.drop(["prices_per_night"], axis = 1), df["prices_per_night"])

# Calculamos Feature Importance
importances = model.feature_importances_

df = pd.DataFrame(data = zip(df.drop(["prices_per_night"], axis = 1).columns, importances),
                              columns = ["Columnas", "Importancia"])

df = df.sort_values("Importancia", ascending = False)

print("Feature Importance:")

for index, (feature, importance) in enumerate(df.values):
    
    print(f"{index + 1:2}. feature {index:2} ({importance:20}): {feature}")

plt.figure(figsize = (12, 8))

plt.title("Feature Importances")
sns.barplot(x = df["Importancia"], y = df["Columnas"], palette = sns.color_palette("husl", 17))

plt.grid()
plt.show()